# python中的序列分类

序列是 python 中重要的协议

按照元素类型是否相同

+ 容器序列：list、tuple、deque
+ 扁平序列：str、bytes、bytearray、array.array

按照元素是否可变

+ 可变类型：list、deque、bytearray、array.array
+ 不可变：str、tuple、bytes

In [3]:
# 元素类型任意
my_list = list()
my_list.append(100)
my_list.append(True)

# 指定元素类型
import array
my_array = array.array('i')
my_array.append(100)
# 初始化数组需要整型，附加字符串抛异常
# my_array.append('abc')
print(my_array)

array('i', [100])


# 序列的abc继承关系

python 中内置的 collections.abc 抽象基类，可以帮助我们理解数据类型实现细节

python 是基于协议的语言，结合鸭子类型和魔法函数，就可以达到实现某种类型

from collections.abc import *

+ Iterable: \_\_iter__
+ Reversible: \_\_reversed__
+ Sized: \_\_len__
+ Container: \_\_contains__
+ Collection: Sized, Iterable, Container
+ Sequence: \_\_getitem__, Reversible, Collection
+ MutableSequence: \_\_setitem__, \_\_delitem__, Sequence

不同魔法函数的组合，构建不同的类型

\_\_all__ = ["Awaitable", "Coroutine",
           "AsyncIterable", "AsyncIterator", "AsyncGenerator",
           "Hashable", "Iterable", "Iterator", "Generator", "Reversible",
           "Sized", "Container", "Callable", "Collection",
           "Set", "MutableSet",
           "Mapping", "MutableMapping",
           "MappingView", "KeysView", "ItemsView", "ValuesView",
           "Sequence", "MutableSequence",
           "ByteString",
           ]




In [ ]:
class Collection(Sized, Iterable, Container):

    __slots__ = ()

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Collection:
            return _check_methods(C,  "__len__", "__iter__", "__contains__")
        return NotImplemented

    
class Reversible(Iterable):

    __slots__ = ()

    @abstractmethod
    def __reversed__(self):
        while False:
            yield None

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Reversible:
            return _check_methods(C, "__reversed__", "__iter__")
        return NotImplemented
    
    
class Sequence(Reversible, Collection):

    """All the operations on a read-only sequence.

    Concrete subclasses must override __new__ or __init__,
    __getitem__, and __len__.
    """

    __slots__ = ()

    @abstractmethod
    def __getitem__(self, index):
        raise IndexError

    def __iter__(self):
        i = 0
        try:
            while True:
                v = self[i]
                yield v
                i += 1
        except IndexError:
            return

    def __contains__(self, value):
        for v in self:
            if v is value or v == value:
                return True
        return False

    def __reversed__(self):
        for i in reversed(range(len(self))):
            yield self[i]

    def index(self, value, start=0, stop=None):
        '''S.index(value, [start, [stop]]) -> integer -- return first index of value.
           Raises ValueError if the value is not present.

           Supporting start and stop arguments is optional, but
           recommended.
        '''
        if start is not None and start < 0:
            start = max(len(self) + start, 0)
        if stop is not None and stop < 0:
            stop += len(self)

        i = start
        while stop is None or i < stop:
            try:
                v = self[i]
                if v is value or v == value:
                    return i
            except IndexError:
                break
            i += 1
        raise ValueError

    def count(self, value):
        'S.count(value) -> integer -- return number of occurrences of value'
        return sum(1 for v in self if v is value or v == value)
    
    
 class MutableSequence(Sequence):

    __slots__ = ()

    """All the operations on a read-write sequence.

    Concrete subclasses must provide __new__ or __init__,
    __getitem__, __setitem__, __delitem__, __len__, and insert().

    """

    @abstractmethod
    def __setitem__(self, index, value):
        raise IndexError

    @abstractmethod
    def __delitem__(self, index):
        raise IndexError

    @abstractmethod
    def insert(self, index, value):
        'S.insert(index, value) -- insert value before index'
        raise IndexError

    def append(self, value):
        'S.append(value) -- append value to the end of the sequence'
        self.insert(len(self), value)

    def clear(self):
        'S.clear() -> None -- remove all items from S'
        try:
            while True:
                self.pop()
        except IndexError:
            pass

    def reverse(self):
        'S.reverse() -- reverse *IN PLACE*'
        n = len(self)
        for i in range(n//2):
            self[i], self[n-i-1] = self[n-i-1], self[i]

    def extend(self, values):
        'S.extend(iterable) -- extend sequence by appending elements from the iterable'
        for v in values:
            self.append(v)

    def pop(self, index=-1):
        '''S.pop([index]) -> item -- remove and return item at index (default last).
           Raise IndexError if list is empty or index is out of range.
        '''
        v = self[index]
        del self[index]
        return v

    def remove(self, value):
        '''S.remove(value) -- remove first occurrence of value.
           Raise ValueError if the value is not present.
        '''
        del self[self.index(value)]

    def __iadd__(self, values):
        self.extend(values)
        return self   

# 序列中+、+=和extend的区别

+ 加号 + 会新生成对象，并且两边类型需要一致
+ 加等 += 就地加，只需要可迭代就行
+ append 附加单个元素，extend 扩展多个元素

In [1]:
# + 加新增
l1 = [1, 2]
l2 = l1 + [3, 4]
print("type(l1): %d, and l1: %s" % (id(l1), l1))
print("type(l2): %d, and l2: %s" % (id(l2), l2))

# += 就地加
l1 += ['3', '4']
l1 += ('5', '6')
l1 += range(2)
print("type(l1): %d, and l1: %s" % (id(l1), l1))


# + 两边类型需相同
# += 只需要可迭代的就行，__iadd__ 魔法函数实现


a = [1, 2]
c = a + [3, 4]
print(c)

a += [3, 4]
print(a)
a += (5, 6)  # 不报错
print(a)
c = a + (7, 8)  # 报错

"""
为什么会有这种差别，
实际上在 += 时，Python
实现的魔法函数是 __iadd__
，调用这个魔法函数实际就是把
+= 右边的值传入 list 的 extend方法。
"""

"""
注意append的方法与extend的方法
append会将参数作为整个。extend会将
参数分开。
"""

type(l1): 140644346027392, and l1: [1, 2]
type(l2): 140644343513344, and l2: [1, 2, 3, 4]
type(l1): 140644346027392, and l1: [1, 2, '3', '4', '5', '6', 0, 1]
[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4, 5, 6]


TypeError: can only concatenate list (not "tuple") to list

# 实现可切片的对象

## 列表切片操作

In [ ]:
'''
模式 [start:end:step]

第一个数字 start 表示切片开始位置，默认 0
第二个数字 end 表示切片截止(但不包含)位置，默认列表长度
第三个数字 step 表示切片的步骤，默认为 1

当 start 为 0 时可以省略
当 end 为列表长度时可以省略
当 step 为 1 时可以省略，并且省略步长时可以同时省略最后一个冒号
当 step 为负数时，表示反向切片，这时 start 应该比 end 的值要大才行
'''

a = [1, 2, 3]
print(id(a))
print(a[::])  # 返回新列表 包含原列表中所有元素
print(a[::-1])  # 返回新列表 原列表的逆排序
print(a[::2])  # 返回新列表 取偶数位
print(a[1::2]) # 返回新列表 取奇数位
print(a[0:2]) # 返回新列表 前开后闭区间
print(a[0:4]) # 返回新列表 结束位置大于列表长度时候 从尾部截断
print(a[100:]) # 返回新列表 切片位置大于列表长度 返回空列表

# 返回新列表 在列表尾部插入新元素 必须是可迭代对象
a[len(a):] = (99,)
a[len(a):] = [99]
print(a)

# 返回新列表 在列表头部插入新元素 必须是可迭代对象
a[:0] = [99]
print(a)

# 返回新列表 新增操作 在列表中间位置插入元素
a[3:3] = [11,12]
print(a)

# 返回新列表 替换操作 0-2的位置替换长度相等的元素
a[:3] = [4,5,6]
print(a)

# 返回新列表 替换操作 0-2的位置替换长度不相等的元素
a[:3] = [4,5,6,7,8]
print(a)

# 返回新列表 替换操作 从3到最后的位置替换长度不相等的元素
a[3:] = [4,5,6,7,8]
print(a)

# 间隔替换元素为0
a[::2] = [0] * int(len(a)/2 + 0.5)
print(a)

#间隔替换元素
a[::2] = ['a','b','c','d']
print(a)

# 删除前三个元素
a[:3] = []
print(a)

# 删除前三个元素
del a[:3]
print(a)

# 间隔删除元素
del a[::2]
print(a)

## 自定义序列类

参考 collections.abc 序列类 Sequence 所需的魔法函数

In [5]:
import numbers

class Group:
    def __init__(self, group_name, company_name, staff):
        self.group_name = group_name
        self.company_name = company_name
        self.staffs = staff

    def __reversed__(self):
        self.staffs.reverse()

    def __getitem__(self, item):
        cls = type(self)
        if isinstance(item, slice):
            return cls(group_name=self.group_name, company_name=self.company_name, staff=self.staffs[item])
        elif isinstance(item, numbers.Integral):
            return cls(group_name=self.group_name, company_name=self.company_name, staff=[self.staffs[item]])

    def __len__(self):
        return len(self.staffs)

    def __iter__(self):
        return iter(self.staffs)

    def __contains__(self, item):
        if item in self.staffs:
            return True
        else:
            return False


staffs = ['linda', 'alex', 'catherine', 'nash', 'curry']
group = Group(group_name='group', company_name='company', staff=staffs)

# 实现getitem后就可实现切片 直接返回list切片对象 如想返回其他对可定制getitem
sub_group = group[0:1]
# if in 语句会调用contains
print('linda' in sub_group)
# 先调用iter,如没实现调用getitem
for item in sub_group:
    print(item)
print(len(sub_group))

True
1


# bisect维护已排序序列

+ bisect 维护一个升序的序列
+ 内部二分查找实现，效率高

In [6]:
import bisect

# 处理已排序 序列 升序
# 内部二分查找算法实现

l1 = list()
bisect.insort(l1, 10)
bisect.insort(l1, 3)
bisect.insort(l1, 2)
bisect.insort(l1, 6)

print(l1)   # [2, 3, 6, 10]

[2, 3, 6, 10]


# 什么时候我们不该用列表

+ 比 list 更好的 python 内置数据结构
+ array 数组 连续的内存空间，性能高
+ deque 双向列表

array 与 list 一个重要区别，array 只能存储指定的数据类型数据

In [7]:
import array

list
my_array = array.array('i')
my_array.append(100)
my_array.append('abc')

TypeError: an integer is required (got type str)

某些应用场景，除了 list 我们还有其他更好的选择

# 列表推导式、生成器表达式和字典推导式

## 列表推导式
列表推导式，或列表生成式，通过一行代码生成列表

In [8]:
# 提取出 1-20 之间的奇数
odd_list = [i for i in range(21) if i % 2 == 1]
print(odd_list)

# 逻辑复杂的情况
def handle_item(item):
    return item * item

odd_list = [handle_item(i) for i in range(21) if i % 2 == 1]
print(odd_list)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
[1, 9, 25, 49, 81, 121, 169, 225, 289, 361]


+ 列表生成式性能高于列表操作
+ 逻辑过于复杂时，列表生成式可读性降低

## 生成器表达式
列表推导 [] -> ()

In [9]:
my_gen = (i for i in range(21) if i % 2 == 1)
print(type(my_gen))     # <class 'generator'>
for i in my_gen:
    print(i)

<class 'generator'>
1
3
5
7
9
11
13
15
17
19


## 字典推导式

In [10]:
d1 = {'key1': 'value1', 'key2': 'value2'}
d2 = {v: k for (k, v) in d1.items()}
print(d2)

{'value1': 'key1', 'value2': 'key2'}


## 集合推导式

In [ ]:
set1 = {v for v in d1.values()}
print(set1)